In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import sys
sys.path.append("..")

In [13]:
from optimus import Optimus

from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, IntegerType, ArrayType

In [14]:
# Create optimus
op = Optimus()

In [15]:
df = op.create.df([
                ("words", "str", True),
                ("num", "int", True),
                ("animals", "str", True),
                ("thing", StringType(), True),
                ("second", "int", True),
                ("filter", StringType(), True)
            ],
            [
                ("  I like     fish  ", 1, "dog dog", "housé", 5 , "a"),
                ("    zombies", 2, "cat", "tv", 6, "b"),
                ("simpsons   cat lady", 2, "frog", "table", 7, "1"),
                (None, 3, "eagle", "glass", 8, "c")
                
            ])

df.show()

+-------------------+---+-------+-----+------+------+
|              words|num|animals|thing|second|filter|
+-------------------+---+-------+-----+------+------+
|  I like     fish  |  1|dog dog|housé|     5|     a|
|            zombies|  2|    cat|   tv|     6|     b|
|simpsons   cat lady|  2|   frog|table|     7|     1|
|               null|  3|  eagle|glass|     8|     c|
+-------------------+---+-------+-----+------+------+



In [16]:
df.dtypes

[('words', 'string'),
 ('num', 'int'),
 ('animals', 'string'),
 ('thing', 'string'),
 ('second', 'int'),
 ('filter', 'string')]

## Append row
### Spark
Not available in Spark. You need to create a dataframe and the union to append a row

In [17]:
df.rows().append(["this is a word",2, "this is an animal", "this is a thing", 64, "this is a filter"]).show()

+-------------------+---+-----------------+---------------+------+----------------+
|              words|num|          animals|          thing|second|          filter|
+-------------------+---+-----------------+---------------+------+----------------+
|  I like     fish  |  1|          dog dog|          housé|     5|               a|
|            zombies|  2|              cat|             tv|     6|               b|
|simpsons   cat lady|  2|             frog|          table|     7|               1|
|               null|  3|            eagle|          glass|     8|               c|
|     this is a word|  2|this is an animal|this is a thing|    64|this is a filter|
+-------------------+---+-----------------+---------------+------+----------------+



## Filter by type
### Spark 
Not available in Spark Vanilla.

### Pandas


In [18]:
df.rows().filter(df["num"]==1).show()

+-------------------+---+-------+-----+------+------+
|              words|num|animals|thing|second|filter|
+-------------------+---+-------+-----+------+------+
|  I like     fish  |  1|dog dog|housé|     5|     a|
+-------------------+---+-------+-----+------+------+



In [ ]:
df.rows().filter_by_dtypes("filter", "integer").show()

## Drop by type

In [10]:
df.rows().drop_by_type("filter", "integer").show()

AttributeError: 'function' object has no attribute 'drop_by_type'

In [11]:
from optimus.functions import is_data_type as fbdt
df.rows().filter_by_dtypes(fbdt("filter", "integer") ).show()

str


AssertionError: The [False] columns do not exists in the DataFrame with the following columns ['words', 'num', 'animals', 'thing', 'second', 'filter']

In [12]:
df.filter(df['num']==1).show()

+-------------------+---+-------+-----+------+------+
|              words|num|animals|thing|second|filter|
+-------------------+---+-------+-----+------+------+
|  I like     fish  |  1|dog dog|housé|     5|     a|
+-------------------+---+-------+-----+------+------+



## Drop 
### Spark
Drop not implemented

In [13]:
df.rows().drop((df["num"]==2) | (df["second"]==5)).show()

+-----+---+-------+-----+------+------+
|words|num|animals|thing|second|filter|
+-----+---+-------+-----+------+------+
| null|  3|  eagle|glass|     8|     c|
+-----+---+-------+-----+------+------+



In [14]:
df.rows().drop_by_dtypes("filter", "int").show()

+-------------------+---+-------+-----+------+------+
|              words|num|animals|thing|second|filter|
+-------------------+---+-------+-----+------+------+
|  I like     fish  |  1|dog dog|housé|     5|     a|
|            zombies|  2|    cat|   tv|     6|     b|
|               null|  3|  eagle|glass|     8|     c|
+-------------------+---+-------+-----+------+------+



In [15]:
from optimus.functions import abstract_udf as audf

def func_data_type(value, attr):
    return value >1


df.rows().drop(audf("num", func_data_type, "bool")).show()

+-------------------+---+-------+-----+------+------+
|              words|num|animals|thing|second|filter|
+-------------------+---+-------+-----+------+------+
|  I like     fish  |  1|dog dog|housé|     5|     a|
+-------------------+---+-------+-----+------+------+

